test3　　演算(t,matmul,sigmoid,relu,elu,leakyrelu,relu6)のテスト

In [1]:
#　colaboraory用: Google drive をマウントする
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# colaboratory用: フォルダを移る
%cd "drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


フォルダは自分の指定のものに変更して下さい。

In [3]:
%cd "ctorch211122/MiniCTorch_Prototype/notebook"

/content/drive/My Drive/Colab Notebooks/ctorch211122/MiniCTorch_Prototype/notebook


ライブラリのimport

In [4]:
! pip install lark-parser

     |████████████████████████████████| 103 kB 5.1 MB/s 


In [5]:
import sys
sys.path.append("../")

import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import minictorch.generator as GN
import minictorch.converter as CV

ニューラルネットの定義 (活性化関数は置き換えて下さい）

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.t = torch.tensor(np.array([[5.0,6,7], [8,9,10]]))
        #self.b = torch.tensor([1.0,2.,3.],dtype=torch.float64)

    def forward(self, x):
        tt=torch.t(self.t)
        z1=torch.matmul(x,tt)
        #z2=torch.sigmoid(z1)
        z2=torch.relu(z1)
        #z2 = torch.tanh(z1)
        #fn = torch.nn.ELU()
        #fn = torch.nn.LeakyReLU()
        #fn = torch.nn.ReLU6()
        #z2 = fn(z1)
        #z2,_= torch.max( z1, 0 )
        #print(z1)
        #print(z2)
        return z2

プロジェクト名、作業フォルダ、JSONファイル名の設定

In [7]:
project = "test3"
folder = "./test"
json_path = folder + '/' + project +'.json'

計算グラフをJSONファイルに出力する

In [8]:
def generate_json( json_path, input_x ):
    model = Net()
    model.eval()
    with torch.no_grad():
        print("[JSON]", json_path )
        GN.generate_minictorch_file( model, input_x, json_path )
    return model

In [9]:
input_to_model = torch.tensor(np.array([[1.0, 2, 3]]), requires_grad=True)

model = generate_json( json_path, input_to_model )

[JSON] ./test/test3.json


JSONファイルをC++コードに変換する

In [10]:
CV.convert_cpp_code( project, folder, model, input_to_model, json_path )

[JSON] ./test/test3.json
[PARAM] ./test/test3_param.cpp
{'name': 'input/x', 'op': 'IO Node', 'in': [], 'output_id': 0, 'shape': [1, 3], 'out': [3], 'sorted_id': 0}
{'name': 'Net/4', 'op': 'prim::Constant', 'in': [], 'output_id': 0, 'shape': [2, 3], 'constant_value': [5.0, 6.0, 7.0, 8.0, 9.0, 10.0], 'out': [2], 'sorted_id': 1}
{'name': 'Net/tt', 'op': 'aten::t', 'in': [1], 'output_id': 0, 'shape': [3, 2], 'out': [3], 'sorted_id': 2}
{'name': 'Net/z1', 'op': 'aten::matmul', 'in': [0, 2], 'output_id': 0, 'shape': [1, 2], 'out': [4], 'sorted_id': 3}
{'name': 'Net/7', 'op': 'aten::relu', 'in': [3], 'output_id': 0, 'shape': [1, 2], 'out': [5], 'sorted_id': 4}
{'name': 'output/output.1', 'op': 'IO Node', 'in': [4], 'output_id': 0, 'shape': [1, 2], 'out': [], 'sorted_id': 5}
[CPP]  ./test/test3.cpp
[MAKE] ./test/Makefile


pythonコードを実行する

In [11]:
# check code
model.train()
y2 = model( input_to_model )
print(y2)
y2.sum().backward()
print( "input", input_to_model.grad )

tensor([[38., 56.]], dtype=torch.float64, grad_fn=<ReluBackward0>)
input tensor([[13., 15., 17.]], dtype=torch.float64)


C++コードをコンパイル、リンクする

In [12]:
!g++ -std=c++14 ./test/test3.cpp ./test/test3_param.cpp -D_NOTEBOOK -I ../../../xtensor -lcblas -o ./test/test3

(注意)  xtensorフォルダにxtensor関連のincludeファイルを置いています。各自の環境に合わせて変更して下さい。

In [13]:
!./test/test3

### forward computation ...
{{ 38.,  56.}}
### backward computation ...
input_grad{{ 13.,  15.,  17.}}
